In [82]:
using WordNet
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators

In [2]:
db = DB("data/corpora/wordnet/")

WordNet.DB

In [3]:
lemma = db['n', "state"]
s1,s2,s3,s4 = synsets(db, lemma)

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


8-element Array{WordNet.Synset,1}:
 (n) province, state (the territory occupied by one of the constituent administrative districts of a nation; "his state is in the deep south")                                                                                                                                                                            
 (n) state (the way something is with respect to its main attributes; "the current state of knowledge"; "his state of health"; "in a weak financial state")                                                                                                                                                               
 (n) state (the group of people comprising the government of a sovereign state; "the state has lowered its income tax")                                                                                                                                                                                                   
 (n) res publica, la

In [5]:
ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model");

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in utf8(::SubString{String}) at ./deprecated.jl:50
 in _collect(::Array{SubString{String},1}, ::Base.Generator{Array{SubString{String},1},Base.#utf8}, ::Base.EltypeUnknown, ::Base.HasShape) at ./array.jl:258
 in #error_content#12(::Symbol, ::String, ::Function, ::LoadError, ::Array{Ptr{Void},1}) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:75
 in execute_request_0x535c5df2(::ZMQ.Socket, ::IJulia.Msg) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:238
 in eventloop(::ZMQ.Socket) at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:143
 in (::IJulia.##24#30)() at ./task.jl:309
while loading /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl, in expression starting on line 31
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in utf8(::SubString{String}) at ./deprecated.jl:50
 in collect_to!(::Array{String,1}, ::Base.Generator{Array{SubString{String},1},Base.#utf8}, ::Int64, ::Int64) at ./array.jl:277
 in _collect(:

In [56]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end
function weighted_average(logprobs, embeddings)
    ret = zeros(first(embeddings))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight.*embedding
    end
    ret
end

function all_logprobs_of_context(embed, context, middle_word)
    [Query.logprob_of_context(embed, context,wv; skip_oov=true, normalise_over_length=true)
        for wv in embed.embedding[middle_word]]
end

function synthesize_embedding(ee,context, middle_word)
    logprobs = all_logprobs_of_context(ee, context,middle_word)
    weighted_average(logprobs, ee.embedding[middle_word])
end

synthesize_embedding (generic function with 1 method)

In [10]:
function poormans_tokenize(source)
    cleaned = filter(s->!ispunct(s), lowercase(source))
    map(string,  split(cleaned))::Vector{SubString{String}}
end

poormans_tokenize (generic function with 1 method)

In [11]:
function lexical_informed_embeddings(db,ee, word)
    source_wvs = ee.embedding[word]
    target_synsets = Base.flatten((haskey(db.lemmas[pos], word) ? synsets(db, db[pos, word]) : WordNet.Synset[] 
        for pos in keys(WordNet.SYNSET_TYPES))) 
        
    synsets4vectors = Dict{eltype(source_wvs), Synset}()
    for synset::Synset in target_synsets
            context::Vector{SubString{String}} = collect(filter!(w->w!=word, poormans_tokenize(synset.gloss)))
            wv = synthesize_embedding(ee,context, word)
            synsets4vectors[wv] = synset
    end
    synsets4vectors
end

lexical_informed_embeddings (generic function with 1 method)

In [95]:
@time lexical_informed_embeddings(db,ee, "solution")

  0.014885 seconds (84.79 k allocations: 1.424 MB)


Dict{Array{Float32,1},WordNet.Synset} with 5 entries:
  Float32[0.0660684,0.208797,0.0588219,-0.0629229,0.0701763,0.0594902,0.0381539,-0.275917,-0.155943,-0.0731264,0.0533249,-0.0456988,-0.123662,-0.0201816,0.0778413,0.00600946,-0.0493329,0.0403639,-0.218606,-0.0292093,-0.0890433,-0.0922866,0.0634917,-0.181567,0.0616838,0.152754,-0.0405237,0.0440584,0.0125958,-0.187449,0.087059,0.00601861,-0.00935936,-0.124986,-0.0650402,0.0569165,-0.00446479,0.0655407,-0.103683,0.0515717,0.0796814,0.155174,0.140808,-0.254336,0.119517,0.0944476,-0.0270894,0.00308659,-0.23041,0.152532,0.110332,0.291473,-0.0467245,0.0775381,0.100264,-0.0106341,-0.142974,-0.0932267,0.0498874,-0.0409321,0.00647375,-0.0184611,-0.0292129,-0.0128562,0.0585012,0.026243,-0.0586521,0.103921,0.00788055,-0.248088,0.143544,-0.0384259,-0.207891,0.100958,0.0538612,0.124087,0.0481915,0.0805845,0.0814414,-0.320794,-0.0808933,-0.0835137,0.141002,-0.0183176,0.0535028,-0.0393761,0.226576,0.1581,0.134159,0.026516,-0.13384,0.0513165,-0.0914

In [1]:
using LightXML

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/LightXML.ji for module LightXML.
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in @windows(::Any, ::Any) at ./deprecated.jl:512
 in include_from_node1(::String) at ./loading.jl:426
 in macro expansion; at ./none:2 [inlined]
 in anonymous at ./<missing>:?
 in eval(::Module, ::Any) at ./boot.jl:234
 in process_options(::Base.JLOptions) at ./client.jl:243
 in _start() at ./client.jl:322
while loading /home/ubuntu/.julia/v0.5/LightXML/src/LightXML.jl, in expression starting on line 19


In [12]:
function load_challenges(xml_file="data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml")
    xdoc = parse_file(xml_file)
    xroot = root(xdoc)
    for text_node in child_elements(xroot)
        #@show text_node
        #text = child_nodes(text_node) |> collect
        #println(text)
        for sentence_node in child_elements(text_node)
            sentence = poormans_tokenize(content(sentence_node))
            
            for token in             
        end
    end
end
load_challenges()

SubString{String}["your","oct","6","editorial","``","the","ill","homeless","referred","to","research","by","us","and","six","of","our","colleagues","that","was","reported","in","the","sept","8","issue","of","the","journal","of","the","american","medical","association"]
